In [1]:
import pandas as pd
import dill
import warnings

from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from feature_engine.encoding import OneHotEncoder
from feature_engine.selection import DropConstantFeatures, DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper
from xgboost import XGBClassifier

from soccer_analytics.data.statsbomb import get_metadata, get_events, load_competition_seasons
from soccer_analytics.data_split import split_by_time
from soccer_analytics.preprocessing import match_list_to_df, AngleNormalizer

In [2]:
competitions = get_metadata()

In [3]:
full_competitions = {
    "1. Bundesliga": ["2015/2016"],
    "FA Women's Super League": ["2018/2019", "2019/2020", "2020/2021"],
    "FIFA World Cup": ["2018", "2022"],
    "Indian Super league": ["2021/2022"],
    "La Liga": ["2015/2016"],
    "Ligue 1": ["2015/2016"],
    "Premier League": ["2015/2016"],
    "Serie A": ["2015/2016"],
    "UEFA Euro": ["2020"],
    "UEFA Women's Euro": ["2022"],
    "Women's World Cup": ["2019", "2023"]
}

In [4]:
matches_by_competition = {}
for competition_name, seasons in full_competitions.items():
    print(competition_name, len(seasons))
    matches_by_competition[competition_name] = load_competition_seasons(
        competition_name, seasons, event_types=["shot"]
    )

1. Bundesliga 1
FA Women's Super League 3
FIFA World Cup 2
Indian Super league 1
La Liga 1
Ligue 1 1
Premier League 1
Serie A 1
UEFA Euro 1
UEFA Women's Euro 1
Women's World Cup 2


In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
class ConsolidateBodyParts(BaseEstimator, TransformerMixin):
    def __init__(self, body_parts_to_keep=["is_body_part_right_foot", "is_body_part_left_foot", "is_body_part_head"]):
        self.body_parts_to_keep = body_parts_to_keep
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        assert "is_body_part_other" in X.columns
        body_parts_to_drop = []
        for column in X.columns:
            if (
                column.startswith("is_body_part") and 
                column not in self.body_parts_to_keep and 
                column != "is_body_part_other"
            ):
                X["is_body_part_other"] = X["is_body_part_other"] | X[column]
                body_parts_to_drop.append(column)
                
        X = X.drop(body_parts_to_drop, axis=1)
        return X
    
class ConsolidateCategories(BaseEstimator, TransformerMixin):
    def __init__(self, column_name, categories_to_keep, other_category_name="other"):
        self.column_name = column_name
        self.categories_to_keep = categories_to_keep
        self.other_category_name = other_category_name
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        other_records = (X[self.column_name].isin(self.categories_to_keep) == False)
        X.loc[other_records, self.column_name] = self.other_category_name
        return X
    
class ConsolidatePositions(BaseEstimator, TransformerMixin):
    """
    Based on Appendix 1: Tactical Positions Guide of the Statsbomb Open Data Specification.
    """
    def __init__(self, column_name="position", other="Unknown"):
        self.column_name = column_name
        self.other = other
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        is_defense = (X[self.column_name].str.endswith("Back")) | (X[self.column_name] == "Goalkeeper")
        is_defensive_midfield = X[self.column_name].str.endswith("Defensive Midfield")
        is_central_midfield = (
            X[self.column_name].str.endswith("Center Midfield") | 
            X[self.column_name].isin(["Right Midfield", "Left Midfield"])
        )
        is_attacking_midfield = X[self.column_name].str.endswith("Attacking Midfield")
        is_forward = (
            X[self.column_name].str.endswith("Wing") |
            X[self.column_name].str.endswith("Striker") |
            X[self.column_name].str.endswith("Forward")
        )
        is_unknown = X[self.column_name] == self.other
        found_positions = (
            is_defense | 
            is_defensive_midfield | is_central_midfield | is_attacking_midfield | 
            is_forward | 
            is_unknown
        )
        unknown_positions = X.loc[found_positions == False, self.column_name]
        if len(unknown_positions) > 0:
            print("Unknown positions:")
            print(unknown_positions)
        X.loc[is_defense, self.column_name] = "Defense"
        X.loc[is_defensive_midfield, self.column_name] = "Defensive Midfield"
        X.loc[is_central_midfield, self.column_name] = "Central Midfield"
        X.loc[is_attacking_midfield, self.column_name] = "Attacking Midfield"
        X.loc[is_forward, self.column_name] = "Forward"
        return X

In [6]:
loss_info = []
model_info = {}
train_data = {}
test_data = {}
for competition_name, matches in matches_by_competition.items():
    print(competition_name)
    train_test_matches, _ = split_by_time(matches, test_frac=0.2)
    train_matches, test_matches = train_test_split(
        train_test_matches, test_size=0.2, random_state=235
    )
    train_df = match_list_to_df(train_matches)
    test_df = match_list_to_df(test_matches)
    train_data[competition_name] = train_df.copy()
    test_data[competition_name] = test_df.copy()
    data_prep_pipeline = Pipeline((
        ("consolidate_techniques", ConsolidateCategories("technique", ["Normal", "Half Volley", "Volley"])),
        ("consolidate_positions", ConsolidatePositions()),
        ("encode_categories", OneHotEncoder(variables=["technique", "position"], drop_last=False)),
        ("consolidate_body_parts", ConsolidateBodyParts()),
        ("drop_unused_columns", DropFeatures(["statsbomb_xg", "coordinates_x", "coordinates_y", "is_goal"])),
        ("drop_constant_columns", DropConstantFeatures()),
        ("create_angle_from_goal", AngleNormalizer(variable="angle_to_goal", new_variable="angle_from_goal")),
        ("passthrough", "passthrough"),
    ))
    # See https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst for parameter
    # definitions
    model = XGBClassifier(
        max_depth=5,
        learning_rate=0.3,
        gamma=0,
        min_child_weight=1,
        subsample=1,
        reg_lambda=1,
        reg_alpha=0,
        eval_metric="mae",
        early_stopping_rounds=5, 
        objective="reg:squarederror",
        monotone_constraints=None,
        random_state=2309
    )
    
    param_grid = {
        "max_depth": [3, 4, 5, 6, 7],
        "learning_rate": [0.3, 0.6],
        "min_child_weight": [1, 5, 10]
    }
    #param_grid = {"max_depth": [5], "learning_rate": [0.3], "min_child_weight": [1]}
    
    model = GridSearchCV(model, param_grid)
    train_df_transformed = data_prep_pipeline.fit_transform(train_df)
    test_df_transformed = data_prep_pipeline.transform(test_df)
    
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        model.fit(
            train_df_transformed, train_df["is_goal"], 
            eval_set=[(test_df_transformed, test_df["is_goal"])],
            verbose=False
        )
        test_probs = model.predict_proba(test_df_transformed)[:,1]
    mae = mean_absolute_error(test_df["is_goal"], test_probs)
    mae_statsbomb = mean_absolute_error(test_df["is_goal"], test_df["statsbomb_xg"])
    mse = mean_squared_error(test_df["is_goal"], test_probs)
    mse_statsbomb = mean_squared_error(test_df["is_goal"], test_df["statsbomb_xg"])
    auroc = roc_auc_score(test_df["is_goal"], test_probs)
    auroc_statsbomb = roc_auc_score(test_df["is_goal"], test_df["statsbomb_xg"])
    loss_info.append({
        "competition": competition_name, 
        "mae": mae,
        "mae_statsbomb": mae_statsbomb,
        "mse": mse,
        "mse_statsbomb": mse_statsbomb,
        "auc": auroc,
        "auc_statsbomb": auroc_statsbomb
    })
    model_info[competition_name] = {"data_prep_pipeline": data_prep_pipeline, "model": model}
    
with open("gbm_feature_engineered.dill", "wb") as f:
    dill.dump(model_info, f)
loss_info = pd.DataFrame(loss_info)

1. Bundesliga
FA Women's Super League
FIFA World Cup
Indian Super league
La Liga
Ligue 1
Premier League
Serie A
UEFA Euro
UEFA Women's Euro
Women's World Cup


In [7]:
loss_info

,competition,mae,mae_statsbomb,mse,mse_statsbomb,auc,auc_statsbomb
0,1. Bundesliga,0.157722,0.144773,0.078462,0.069194,0.809068,0.859355
1,FA Women's Super League,0.178092,0.165271,0.094291,0.088185,0.746624,0.783300
2,FIFA World Cup,0.145763,0.134978,0.071913,0.064855,0.758164,0.816374
3,Indian Super league,0.178392,0.151895,0.087013,0.081117,0.795558,0.811360
4,La Liga,0.167414,0.151398,0.084819,0.076159,0.792532,0.837254
5,Ligue 1,0.163017,0.145112,0.091958,0.077408,0.716438,0.785992
6,Premier League,0.151437,0.142464,0.077429,0.070332,0.774820,0.825444
7,Serie A,0.141058,0.134125,0.075461,0.069914,0.766181,0.819325
8,UEFA Euro,0.179082,0.172280,0.095366,0.092162,0.760690,0.856552
9,UEFA Women's Euro,0.136749,0.121463,0.059141,0.050070,0.745614,0.844055
